In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import silhouette_samples, silhouette_score, v_measure_score
from sklearn.datasets import load_files
import re
from unidecode import unidecode
from mpl_toolkits import mplot3d
from yellowbrick.cluster import KElbowVisualizer
print("Bibliotecas Baixadas")


nltk.download('rslp')
nltk.download('stopwords')


Bibliotecas Baixadas


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caios\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Carregue os DataFrames origem_ID e text do CSV
caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )
origem_ID['ID'] = range(1, len(origem_ID) + 1)

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )
text['ID'] = range(1, len(text) + 1)


In [ ]:
#Limpando text.txt

text['dsc_item'] = text['dsc_item'].astype(str)
origem_ID['classif'] = origem_ID['classif'].astype(str)
text["dsc_item"] = text["dsc_item"].apply(lambda x: x.lower()) #converte todas as letras para minúsculo
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: x.lower()) #converte todas as letras para minúsculo 
text["dsc_item"] = text["dsc_item"].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
text["dsc_item"] = text["dsc_item"].apply(lambda x: unidecode(x)) #remove acentos
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: unidecode(x)) #remove acentos

stemmer = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
text["dsc_item"] = text["dsc_item"].apply(lambda x: stemmer.stem(x))

stemmer1 = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: stemmer.stem(x))


# Função para encontrar o correspondente com o maior score em 'classif' para um valor de 'dsc_item'
def find_best_match(dsc_item_value):
    matches = process.extractOne(dsc_item_value, origem_ID['classif'], scorer=fuzz.ratio)
    return matches


# Encontre o correspondente com o maior score em 'classif' para cada valor único em 'dsc_item'
unique_dsc_item_values = text['dsc_item'].unique()
best_matches = {}

for dsc_item_value in unique_dsc_item_values:
    best_match = find_best_match(dsc_item_value)
    best_matches[dsc_item_value] = best_match

# Crie um DataFrame com os resultados, incluindo as colunas 'ID_classif' e 'ID_dsc_item'
#result_df = pd.DataFrame(list(best_matches.values()), index=list(best_matches.keys()), columns=['melhor_classif', 'score'])
result_df = pd.DataFrame(list(best_matches.values()), index=list(best_matches.keys()), columns=['melhor_classif', 'score', 'additional_identifier'])

result_df['dsc_item'] = result_df.index
result_df.reset_index(drop=True, inplace=True)

# Adicione as colunas 'ID_classif' e 'ID_dsc_item'
result_df['ID_classif'] = origem_ID.set_index('classif').loc[result_df['melhor_classif'], 'ID'].values
result_df['ID_dsc_item'] = text.set_index('dsc_item').loc[result_df['dsc_item'], 'ID'].values

# Reordene as colunas para ter 'dsc_item' na primeira posição
result_df = result_df[['dsc_item', 'melhor_classif', 'score', 'ID_classif', 'ID_dsc_item']]

# Salvar o DataFrame resultante em um novo CSV
result_df.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(result_df.head())


In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import nltk
from sklearn.metrics import silhouette_samples, silhouette_score, v_measure_score
from sklearn.datasets import load_files
import re
from unidecode import unidecode
from mpl_toolkits import mplot3d
from yellowbrick.cluster import KElbowVisualizer
print("Bibliotecas Baixadas")


nltk.download('rslp')
nltk.download('stopwords')

# Carregue os DataFrames origem_ID e text do CSV
caminho_arquivo_1= 'cod_material_servico.txt'
origem_ID = pd.read_csv(caminho_arquivo_1,sep=";" )
origem_ID['ID'] = range(1, len(origem_ID) + 1)

caminho_arquivo_2= 'text.txt'
text = pd.read_csv(caminho_arquivo_2,sep="," )
text['ID'] = range(1, len(text) + 1)


text['dsc_item'] = text['dsc_item'].astype(str)
origem_ID['classif'] = origem_ID['classif'].astype(str)
text["dsc_item"] = text["dsc_item"].apply(lambda x: x.lower()) #converte todas as letras para minúsculo
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: x.lower()) #converte todas as letras para minúsculo 
text["dsc_item"] = text["dsc_item"].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: re.sub('|,|\.|/|$|\(|\)|-|\+|:|•', '', x)) #remove números e caracteres especiais 
text["dsc_item"] = text["dsc_item"].apply(lambda x: unidecode(x)) #remove acentos
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: unidecode(x)) #remove acentos

stemmer = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
text["dsc_item"] = text["dsc_item"].apply(lambda x: stemmer.stem(x))

stemmer1 = nltk.stem.RSLPStemmer() #converte as palavras para seu radical 
origem_ID['classif'] = origem_ID['classif'].apply(lambda x: stemmer.stem(x))


# Função para encontrar o correspondente com o maior score em 'classif' para um valor de 'dsc_item'
def find_best_match(dsc_item_value):
    matches = process.extractOne(dsc_item_value, origem_ID['classif'], scorer=fuzz.ratio)
    return matches

# Encontre o correspondente com o maior score em 'classif' para cada valor único em 'dsc_item'
unique_dsc_item_values = text['dsc_item'].unique()
best_matches = {}

for dsc_item_value in unique_dsc_item_values:
    best_match = find_best_match(dsc_item_value)
    best_matches[dsc_item_value] = best_match

# Crie um DataFrame com os resultados, incluindo as colunas 'ID_classif' e 'ID_dsc_item'
result_df = pd.DataFrame(list(best_matches.values()), index=list(best_matches.keys()), columns=['melhor_classif', 'score'])
result_df['dsc_item'] = result_df.index
result_df.reset_index(drop=True, inplace=True)

# Adicione as colunas 'ID_classif' e 'ID_dsc_item'
result_df['ID_classif'] = origem_ID.set_index('classif').loc[result_df['melhor_classif'], 'ID'].values
result_df['ID_dsc_item'] = text.set_index('dsc_item').loc[result_df['dsc_item'], 'ID'].values

# Reordene as colunas para ter 'dsc_item' na primeira posição
result_df = result_df[['dsc_item', 'melhor_classif', 'score', 'ID_classif', 'ID_dsc_item']]

# Salvar o DataFrame resultante em um novo CSV
result_df.to_csv('resultado.csv', index=False)

# Exibir as primeiras linhas do DataFrame resultante
print(result_df.head())